In [1]:
import numpy as np
import numpy.linalg as lin
import pickle as pickle
import matplotlib.pyplot as plt
file_header = 'samples_episodes/samples_random_policy_small.pickle'
with open(file_header+'1', 'rb') as handle:
    samples = pickle.load(handle)


In [8]:
bk_color = 144
image_width, image_height = samples['states'][0][0].shape
bk_image = np.ones((image_width, image_height), dtype = int) * 144
def phi_sum_row(img):
    ps = np.zeros((image_height))
    for r in range(image_height):
        if np.max(img[r, :]) == 0:
            feat = np.min(img[r, :])
        else:
            feat = np.max(img[r, :])
        ps[r] = feat
    return ps


def phi_sum_col(img):
    ps = np.zeros((image_width))
    for c in range(image_width):
        if np.max(img[:, c]) == 0:
            feat = np.min(img[:, c])
        else:
            feat = np.max(img[:, c])
        ps[c] = feat
    return ps

def phi_sum(img):
    return np.hstack([phi_sum_row(img), phi_sum_col(img)])

In [12]:


num_samples = 0
for i in range(len(samples['states'])):
    num_samples += len(samples['states'][i])

num_features = image_width + image_height  
Phi = np.zeros((num_samples, num_features), dtype=np.float32)
Phi_next = np.zeros((num_samples, num_features), dtype=np.float32)
A = np.zeros(num_samples, dtype=np.int)
R = np.zeros(num_samples, dtype=np.float32)
index = 0
for epi in range(len(samples['states'])):
    for t in range(len(samples['states'][epi])):
        s = samples['states'][epi][t]
        Phi[index] = phi_sum(s)
        A[index] = samples['actions'][epi][t]
        R[index] = samples['rewards'][epi][t]
        if R[index] == 0:#episode is not ending
            Phi_next[index] = phi_sum(samples['states'][epi][t+1])
        else:
            Phi_next[index] = np.zeros(num_features, dtype=np.float32)
    

In [21]:
print(Phi_next.shape)
print(Phi.shape)
print(np.unique(A))
A[A==2] = 1
print(np.unique(A))
num_actions = len(np.unique(A))
print(num_actions)



(2359, 160)
(2359, 160)
[0 1]
[0 1]
2


In [27]:

# https://drive.google.com/file/d/0B4yKRDF2fvC8S0hBeUlrX1lDa2c/view
def ls_model():
    assert(Phi.shape[0] ==  len(A))
    assert(len(A) == len(R))
    assert(Phi.shape == Phi_next.shape)
    N = Phi.shape[0]
    H = np.zeros((num_actions, num_features, num_features))
    E = np.zeros((num_actions, num_features, num_features))
    e = np.zeros((num_actions, num_features))
    for a in range(num_actions):
        E[a] = np.eye(num_features) * 0.01 
    
    for i in range(N):
        ps = Phi[i]
        a = A[i]
        psnext = Phi_next[i]
        H[a] += ps * ps.T
        E[a] += psnext * ps.T
        e[a] += ps * R[i]
    F = np.zeros((num_actions, num_features, num_features))
    f = np.zeros((num_actions, num_features))
    for a in range(num_actions):
        F[a] = E[a] * np.linalg.pinv(H[a])
        f[a] = np.linalg.solve(E[a], e[a])
    return F, f
Fa, fa = ls_model()
print(Fa.shape)
print(fa.shape)

(2, 160, 160)
(2, 160)


In [38]:

gamma = 0
def lam_lstd(F, f, Phi):
    w = np.random.randn(num_features)
    A = np.eye(num_features, dtype=np.float32)
    b = np.zeros(num_features, dtype=np.float32)
    for ps in Phi:
        Qmax = 0.0
        abest = -1
        for a in range(F.shape[0]):
            print(f[a].dot(ps).shape)
            print(w.shape)
            Q = f[a].dot(ps) + gamma * w.dot(np.matmul(F[a], ps))
            if Q > Qmax:
                Qmax = Q
                abest = a
        phi_next = F[abest] * ps
        r = f[a].dot(ps)
        A += ps * (gamma*phi_next - ps)
        b += ps * r
    w = - np.linalg.solve(A, b)
    return w

In [37]:
w_lam = lam_lstd(Fa, fa, Phi)

()
(2,)


ValueError: shapes (2,) and (160,) not aligned: 2 (dim 0) != 160 (dim 0)